In [1]:
import os

os.environ['R_HOME'] = '/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/R'

In [2]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


In [3]:
sc_ad = sc.read_h5ad('/home/comp/cszrwang/data/osmfish/sc_osm.h5ad')
st_ad = sc.read_h5ad('/home/comp/cszrwang/data/osmfish/st_osm.h5ad')

sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

In [4]:
sc_ad.obs['celltype']

F2S4_160722_001_A01              Vip
F2S4_160722_001_B01              Vip
F2S4_160722_001_C01        L5 NP CTX
F2S4_160722_001_D01      L4/5 IT CTX
F2S4_160722_001_E01              Sst
                           ...      
F2S4_181129_012_D01      L4/5 IT CTX
F2S4_181129_012_E01    L2/3 IT CTX-1
F2S4_181129_012_F01    L2/3 IT CTX-1
F2S4_181129_012_G01      L4/5 IT CTX
F2S4_181129_012_H01    L2/3 IT CTX-1
Name: celltype, Length: 5392, dtype: category
Categories (11, object): ['L2/3 IT CTX-1', 'L4/5 IT CTX', 'L5 IT CTX', 'L5 NP CTX', ..., 'L6b CTX', 'Lamp5', 'Sst', 'Vip']

In [5]:
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='celltype',deg_method='t-test',sm_size=100000,use_gpu=True)

Setting global seed: 42
normalizing counts per cell
    finished (0:00:00)
normalizing counts per cell
    finished (0:00:00)
### Finding marker genes...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
celltype
L6 IT CTX        12
Vip               9
L5 IT CTX         8
L6 CT CTX         8
L6b CTX           8
L5 PT CTX         6
Lamp5             6
Sst               6
L2/3 IT CTX-1     5
L5 NP CTX         5
L4/5 IT CTX       4
Name: count, dtype: int64
### Used gene numbers: 27
### Initializing sample probability
### Genetating simulated spatial data using scRNA data with mode: unbalance
### Genetating simulated spatial data using scRNA data with mode: sqr

In [6]:
spoint_model.train(max_steps=5000, batch_size=1024)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:47<00:00,  2.30it/s, loss=56.1, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:47<00:00,  2.09it/s, loss=56.1, v_num=1]


Step 5000: Test inference loss=-0.827: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [26:04<00:00,  3.20it/s]


In [7]:
pre = spoint_model.deconv_spatial()

In [8]:
pre

,L4/5 IT CTX,L2/3 IT CTX-1,L6 IT CTX,L6b CTX,L6 CT CTX,Vip,L5 IT CTX,L5 NP CTX,Lamp5,Sst,L5 PT CTX
400_27600,0.000000,0.000000,0.000000,0.614960,0.000000,0.000000,0.018927,0.263828,0.000000,0.083419,0.000000
400_28400,0.000000,0.000000,0.073008,0.272808,0.032709,0.101838,0.070577,0.326145,0.000000,0.117198,0.000000
400_29200,0.000000,0.000000,0.120079,0.486872,0.017581,0.068789,0.079357,0.100509,0.000000,0.124220,0.000000
400_30000,0.010734,0.000000,0.011383,0.295445,0.085279,0.150979,0.109916,0.294775,0.000000,0.024704,0.000000
400_30800,0.000000,0.000000,0.000000,0.255340,0.016824,0.028511,0.084095,0.593992,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
22800_34000,0.191567,0.098156,0.014083,0.261966,0.036161,0.060540,0.089702,0.073568,0.041807,0.013941,0.118509
22800_34800,0.029986,0.049019,0.176367,0.194773,0.081687,0.167577,0.102780,0.059392,0.098221,0.028987,0.011211
22800_35600,0.084392,0.080134,0.039309,0.171902,0.168909,0.158903,0.072166,0.057551,0.088849,0.021165,0.056721
22800_37200,0.000000,0.000000,0.000000,0.474036,0.000000,0.029162,0.032877,0.384955,0.000000,0.056823,0.000000


In [14]:
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/osmFISH//SPACEL/"

In [15]:
pre.to_csv(output_file_path + '/SPACEL_result.csv')

In [12]:
st_ad_show = spoint_model.st_ad

In [13]:
st_ad_show

AnnData object with n_obs × n_vars = 737 × 27
    obs: 'index', 'x', 'y', 'n_genes', 'batch', 'L4/5 IT CTX', 'L2/3 IT CTX-1', 'L6 IT CTX', 'L6b CTX', 'L6 CT CTX', 'Vip', 'L5 IT CTX', 'L5 NP CTX', 'Lamp5', 'Sst', 'L5 PT CTX'
    var: 'n_cells'
    uns: 'celltypes'
    layers: 'norm'